In [2]:
import MetaTrader5 as mt5
import time
import pandas as pd
from IPython.display import clear_output



In [3]:
# Nos conectamos con MTD5
if not mt5.initialize():
    print("error en la conexion")
    exit()
else:
    print("conexion exitosa")
    print(mt5.account_info())

conexion exitosa
AccountInfo(login=87779686, trade_mode=0, leverage=100, limit_orders=200, margin_so_mode=0, trade_allowed=True, trade_expert=True, margin_mode=2, currency_digits=2, fifo_close=False, balance=99999.63, credit=0.0, profit=0.0, equity=99999.63, margin=0.0, margin_free=99999.63, margin_level=0.0, margin_so_call=50.0, margin_so_so=30.0, margin_initial=0.0, margin_maintenance=0.0, assets=0.0, liabilities=0.0, commission_blocked=0.0, name='Mariano Orozco', server='MetaQuotes-Demo', currency='USD', company='MetaQuotes Ltd.')


In [4]:
# Función para enviar órdenes de compra
def buy(symbol, volume):
    try:
        sl = mt5.symbol_info_tick(symbol).ask * 0.15 
        tp = mt5.symbol_info_tick(symbol).ask * 10
        order_type = mt5.ORDER_TYPE_BUY
        request = {
            "action": mt5.TRADE_ACTION_DEAL,
            "symbol": symbol,
            "volume": volume,
            "type": order_type,
            "price": mt5.symbol_info_tick(symbol).ask,
            "sl": sl,
            "tp": tp,
            "deviation": 10,
            "magic": 123456,
            "comment": "Buy order",
            "type_time": mt5.ORDER_TIME_GTC,
            "type_filling": mt5.ORDER_FILLING_IOC,
        }
        result = mt5.order_send(request)
        return result
    except Exception as buy:
        print(buy)
# probando

symbol = "EURUSD"
volume = 0.1  # Ajusta el volumen según tus necesidades

previous_signal = None  # Variable para almacenar el estado anteri
result = buy(symbol, volume)
if result is None:
    print ( mt5.last_error())

In [8]:
def finish_order():
    # Obtener todas las posiciones abiertas
    positions = mt5.positions_get()
    print(position)
    if positions is None:
        print("No hay posiciones abiertas.")
    else:
        for position in positions:
            # Imprimir información de cada posición
            print(f"Símbolo: {position.symbol}, Volumen: {position.volume}, Tipo: {'Compra' if position.type == 0 else 'Venta'}")

            # Si deseas cerrar una posición de venta
            if position.type == 1:  # 1 es para ventas
                request = {
                    "action": mt5.TRADE_ACTION_DEAL,
                    "symbol": position.symbol,
                    "volume": position.volume,
                    "type": mt5.ORDER_TYPE_BUY,  # Para cerrar una venta, envías una compra
                    "position": position.ticket,  # Ticket de la posición que deseas cerrar
                    "deviation": 10,
                    "magic": 123456,
                    "comment": "Cierre de operación",
                    "type_time": mt5.ORDER_TIME_GTC,
                    "type_filling": mt5.ORDER_FILLING_IOC,
                }

                result = mt5.order_send(request)

                if result is None:
                    print("Error al enviar la orden de cierre:", mt5.last_error())
                else:
                    print("Operación cerrada con éxito:", result)
finish_order()


UnboundLocalError: cannot access local variable 'position' where it is not associated with a value

In [6]:

symbol = "EURUSD"
volume = 0.1  # Ajusta el volumen según tus necesidades

previous_signal = None  # Variable para almacenar el estado anterior


In [7]:
while True:
    # Obtiene los datos en tiempo real
    rates = mt5.copy_rates_from_pos(symbol, mt5.TIMEFRAME_M1, 0, 1000)
    if rates is None:
        print("Error al obtener datos.")
        break
    
    # Convierte los datos a un DataFrame
    data = pd.DataFrame(rates)
    data['time'] = pd.to_datetime(data['time'], unit='s')
    data.set_index('time', inplace=True)

    # Calcula las medias móviles
    data['SMA_14'] = data['close'].rolling(window=14).mean()
    data['SMA_50'] = data['close'].rolling(window=50).mean()


    # Extraemos los datos para verificar 
    # data.to_excel("probando.xlsx", engine="openpyxl")

    # Obtiene las últimas medias móviles
    last_sma_14 = data['SMA_14'].iloc[-1]
    last_sma_50 = data['SMA_50'].iloc[-1]

    # imprimimos los datos obtenidos
    print("medida 14", last_sma_14)

    print("Medida 50", last_sma_50)
    # Determina el cruce
    
    
    
    ######  Apenas ejecutemos el script corroborar si el precio esta en posicion de compra o venta
    # en el caso de que este en posicion de compra no tocar nada y en el caso de que este en posicion
    # de venta cambiar el valor del previous_signal a "sell" para que no realice ninguna venta
    if previous_signal is None:
        previous_signal = 'none'
    
    if last_sma_14 > last_sma_50 and previous_signal != 'buy':
        print("Cruce de compra detectado. Realizando compra...")
        buy(symbol, volume)
        previous_signal = 'buy'

        
    elif last_sma_14 < last_sma_50 and previous_signal != 'sell':
        print("Finalizando compras")
        finish_order()
        previous_signal = 'sell'

    time.sleep(2)
        # obtenemos el saldo actua
    print("\nEn transaccion")

    # tiempo de espera antes de volver a comprobar
    time.sleep(10)
    clear_output(wait=True)


# Cierra la conexión con MetaTrader 5
mt5.shutdown()


medida 14 1.0872035714285715
Medida 50 1.0872036
Finalizando compras


UnboundLocalError: cannot access local variable 'position' where it is not associated with a value